# From raw measurements to compressed sensing measurements
    Now we have generated raw FLIM measurements from generate_raw_measurement, we can use the raw measurements to simulate the compressively sensed measurements.

In [ ]:
# Parameters
sample_size = 8000
image_size = 128

## Some preparation

In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
from utilities import draw
import h5py

A function to read in raw_measurement files into the memory

In [ ]:
# This function reads the raw measurements data from the Raw_Measurements file
# Parameters:
# - starting_index is where the index starts
# - batch is the number of batches
# For example, read_raw_data(20,10) will return the raw measurement data from sample_20 to sample_30
# There are up to 8000 raw measurements data
def read_raw_data(starting_index, batch):
    raw_measurements = np.zeros([batch,128,128,256]).astype(np.uint8)
    for j in range(batch):
        i = j+starting_index
        raw_measurements[j,:,:] = scipy.io.loadmat('Data\Raw_Measurements\sample_'+str(i)+'.mat')['measurements']
    return raw_measurements

## Generate images
Compressive measurements are made by $y = \phi x$, where y is the compressively measured datasets, and x is the original image. $\phi$ is the measurement matrix
x in here is a 1D array, so we reshape the lifetime raw measurements into 

$x = [batch_i, t, y]$

So first of all we need a Hadamard Matrix, as our measurement matrix $\phi$
### Generate hadamard patterns

In [ ]:
# raw_measurements = [batch_i, x, y, t]
hadamard= scipy.linalg.hadamard(image_size*image_size)
print('The shape of hadamard matrix is ', hadamard.shape)
plt.imshow(hadamard[:128,:128])

In [ ]:
# Get the hadamard order
#with h5py.File('pattern_index_HR_128.mat','r') as f:
#    hadamard_idx = np.squeeze((np.array(f['pat_idx'])-1).astype(int))
# Reorder hadamard according to the order
# Only keeping 1800 of hadamard matrix, compression of 10%
#hadamard_ordered = hadamard[hadamard_idx[:1800],:]

### Read in raw measurements
 We have previously generated 8000 raw measurements samples, so there's where the 8000 comes from. 
 Future sample data might be bigger or smaller

In [ ]:
# Use batch size of multiples of 8000 to use all samples
def batch_measurements(batch_size):
    total_batch = sample_size//batch_size
    for batch in range(total_batch):
        # read 1 data per batch, starting from 0
        raw_measurements = read_raw_data(batch*batch_size ,batch_size)
        # Raw data is in format of [batch_i,x,y,t]
        # we need [batch_i,t,pixel] to Apply hadamard matrix
        x_2D = np.moveaxis(raw_measurements,[1,2,3],[2,3,1])
        # x_2D = [batch_i,t,x,y]
        x = np.reshape(x_2D, (x_2D.shape[0],x_2D.shape[1],-1))
        # x = [batch_i,t,pixel_i]
        yield x

## Apply hadamard pattern to the images
CPU based computation

In [ ]:
# Allocate some memories 
cs_data = np.zeros((sample_size,256,image_size*image_size))

In [ ]:
batch_size = 1
for i, x in enumerate(batch_measurements(batch_size)):
    #cs_data[i*batch_size:(i+1)*batch_size,:,:] = np.dot(x,hadamard_ordered.T)
    cs_data[i*batch_size:(i+1)*batch_size,:,:] = np.dot(x,hadamard.T)

In [ ]:
CS_measurements.shape

 Alternatively, GPU vectorised based computation based on cupy

In [ ]:
import cupy as cp
import tqdm

In [ ]:
# Allocate some memories 
cs_data = np.zeros((sample_size,256,image_size*image_size))

In [ ]:
batch_size = 100
for i, x in enumerate(tqdm.tqdm(batch_measurements(batch_size))):
    # load into gpu memory
    x_gpu = cp.asarray(x)
    hadamard_gpu = cp.asarray(hadamard_ordered.T)

    # matrix dot product
    CS_measurements_gpu = cp.dot(x_gpu, hadamard_gpu)

    # Move back to host memory
    cs_data[i*batch_size : (i+1)*batch_size] = cp.asnumpy(CS_measurements_gpu)

In [ ]:
# Write into files
with h5py.File('Data/cs_measurements.h5','w') as f:
    f.create_dataset('cs_data', data=cs_data)
# draw(characters[:4])

# Writing into .mat
Now we have all the data, we just need to make all files into the format that's required by the CNN
The format takes as sample_int.mat, the mat is as 
{'cs_data': [], 'intensity_image':[], 'lifetime_image': []}

In [2]:
def get_data(idx):
    f_gt = h5py.File('Data/Ground_Truth/ground_truth.h5','r')
    f_cs = h5py.File('Data/CS_Measurements/cs_measurements.h5','r')
    intensity_image = np.array(f_gt['intensity_ground_truth'][idx])
    lifetime_image = np.array(f_gt['lifetime_ground_truth'][idx])
    cs_data = np.array(f_cs['cs_data'][idx])
    f_gt.close()
    f_cs.close()
    return cs_data, intensity_image, lifetime_image

In [ ]:
def write_mat_fromfile(idx):
    cs_data, intensity_image, lifetime_image = get_data(idx)
    scipy.io.savemat(('Data/training_data/sample_'+str(idx)+'.mat'), {'cs_data': cs_data, 'intensity_image': intensity_image, 'lifetime_image': lifetime_image})

In [7]:
from tqdm import tqdm
for i in tqdm(range(sample_size)):
    write_mat(i)

100%|██████████████████████████████████████████████████████████████████████████████| 8000/8000 [06:03<00:00, 22.00it/s]
